# STAT413 Homework 5
Lauren Huang

## Problem 2
Instructions: Write a brief explanation of the code and show your results. You can explore the code by varying the design parameters.

In [1]:
! pip install torch numpy transformers datasets tiktoken wandb tqdm

In [2]:
! python data/shakespeare_char/prepare.py

length of dataset in characters: 1,115,394
all the unique characters: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size: 65
train has 1,003,854 tokens
val has 111,540 tokens


I do not have a GPU, so I will be following this set of instructions.
I have altered the parameters from 20 eval_iters to 40, and from a block size of 64 to 256.

In [3]:
! python train.py config/train_shakespeare_char.py --device=cpu --compile=False --eval_iters=40 --log_interval=1 --block_size=256 --batch_size=12 --n_layer=4 --n_head=4 --n_embd=128 --max_iters=2000 --lr_decay_iters=2000 --dropout=0.0

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 200
log_interval = 10 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 64
block_size = 256 # context of up to 256 previous characters

# baby GPT model :)
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 5000
lr_decay_iters = 5000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of 

c:\Users\cupca\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\amp\grad_scaler.py:131: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.

0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [4]:
! python sample.py --out_dir=out-shakespeare-char --device=cpu

Overriding: out_dir = out-shakespeare-char
Overriding: device = cpu
number of parameters: 0.80M
Loading meta from data\shakespeare_char\meta.pkl...

Then all other the dreathers a shell sprise,
Sorng from the encesss of the bembroush of hean?

LADY:
It the shall to fie it the post away divenge,
You save a shoop sing that bear in them,
Find Did and these of my lord.
Thou great I thou gosterforditables?

Thou conquer:
But shy had look it, your notess thee plose?

COMINIUS:
That is true your makes lady! that will grest but
That some me frier more me chargess with own thee,
Telch his he't were ofter, thee guil do the quor offece's
And mored ange
---------------

A commegent yet here hope, chear me at Partsenen,
Whose to are the feel deash of his speak the gend
Thou sterve's the peace; and sleep that artith,
I'll of more heavent in fring and tongure,
And nothing the for this kind the plating a
even follok'd their buth resenger, and Enger and
The waver of friely of and the their Ediffury;
An

In [5]:
! python data/openwebtext/prepare.py

c:\Users\cupca\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\cupca\.cache\huggingface\hub\datasets--openwebtext. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\cupca\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_downloa

Unfortunately, the GPU on my device is not compatible, so I am unable to run the code below.

In [6]:
! torchrun --standalone --nproc_per_node=8 train.py config/train_gpt2.py

^C


In [7]:
# Run on the first (master) node with example IP 123.456.123.456:
! torchrun --nproc_per_node=8 --nnodes=2 --node_rank=0 --master_addr=123.456.123.456 --master_port=1234 train.py
# Run on the worker node:
! torchrun --nproc_per_node=8 --nnodes=2 --node_rank=1 --master_addr=123.456.123.456 --master_port=1234 train.py

Overriding config with config/train_gpt2.py:
Overriding config with config/train_gpt2.py:
# config for training GPT-2 (124M) down to very nice loss of ~2.85 on 1 node of 8X A100 40GB
# launch as the following (e.g. in a screen session) and wait ~5 days:
# $ torchrun --standalone --nproc_per_node=8 train.py config/train_gpt2.py

wandb_log = True
wandb_project = 'owt'
wandb_run_name='gpt2-124M'

# these make the total batch size be ~0.5M
# 12 batch size * 1024 block size * 5 gradaccum * 8 GPUs = 491,520
batch_size = 12
block_size = 1024
gradient_accumulation_steps = 5 * 8

# this makes total number of tokens be 300B
max_iters = 600000
lr_decay_iters = 600000

# eval stuff
eval_interval = 1000
eval_iters = 200
log_interval = 10

# weight decay
weight_decay = 1e-1

# config for training GPT-2 (124M) down to very nice loss of ~2.85 on 1 node of 8X A100 40GB
# launch as the following (e.g. in a screen session) and wait ~5 days:
# $ torchrun --standalone --nproc_per_node=8 train.py config/tra

W1105 23:33:01.738000 16048 torch\distributed\elastic\multiprocessing\redirects.py:27] NOTE: Redirects are currently not supported in Windows or MacOs.
W1105 23:33:01.799000 16048 torch\distributed\run.py:757] 
W1105 23:33:01.799000 16048 torch\distributed\run.py:757] *****************************************
W1105 23:33:01.799000 16048 torch\distributed\run.py:757] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1105 23:33:01.799000 16048 torch\distributed\run.py:757] *****************************************
C:\Users\cupca\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\distributed\distributed_c10d.py:613: UserWarning: Attempted to get default timeout for nccl backend, but NCCL support is not compiled
  warnings.warn("Attempted to get default timeout for nccl backend, but NCCL support is not compiled")
C:\Us